# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
!pip install langchain_community langchain_experimental langchain_openai

In [13]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [14]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [15]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [ ]:
# import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

# SERPAPI_API_KEY  = os.getenv('SERPAPI_API_KEY')

In [16]:
%pip install faiss-cpu

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Note: you may need to restart the kernel to use updated packages.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [17]:
%pip install google-search-results

Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [21]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [10]:
OBJECTIVE = "Write a weather report for SF today"

In [11]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [ ]:
# import os
# os.environ["SERPAPI_API_KEY"] = "3e655cab8c4a95ed2a8daf8806c725874cede8bad9a789bad5c1ed99a6525f27"

In [ ]:
# import os

# # Set your valid SerpAPI key here
# os.environ["SERPAPI_API_KEY"] = "3e655cab8c4a95ed2a8daf8806c725874cede8bad9a789bad5c1ed99a6525f27"

baby_agi({"objective": OBJECTIVE})

C:\Users\happy\AppData\Local\Temp\ipykernel_114064\3382780125.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should use the TODO action to create a todo list
Action: TODO
Action Input: "Create a todo list for writing a weather report for SF today"
Observation: 

1. Gather all necessary information: Start by collecting all the necessary information for writing a weather report for San Francisco today. This includes current temperature, humidity, wind speed, and any potential weather events.

2. Check reliable sources: Make sure to check reliable sources such as the National Weather Service or local news stations for accurate and up-to-date information.

3. Determine the format: Decide on the format for the weather report. Will it be a written report, a video, or a combination of both? This will help guide the rest of the process.

4. Create an outline: Before diving into writing, create an outline for the weather report. This will help organize the information 

{'objective': 'Write a weather report for SF today'}

Changing prompt and objective

In [18]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
# vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [21]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a senior travel-logistics specialist.  "
    "Your goal is to build a comprehensive preparation checklist—from gathering documents through packing list—for this trip:\n\n"
    "{objective}\n\n"
    "Produce a step-by-step, prioritized task list (numbered)."
)

todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are a Travel-Preparation AI that helps plan every step for a complex trip.
Objective: {objective}
Previously completed tasks & findings:
{context}

Use only the tools available to you to accomplish the next task."""
suffix = """Next task: {task}

Begin thinking (you can call tools like Search or TODO):
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [22]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True,handle_parsing_errors=True       
)

In [23]:
OBJECTIVE = "Create a preparation list From documents needed down to luggage items for our July 27 to August 7, 2025 Trip to Peru. We will be visiting Lima, Cusco, Machu Picchu, Rainbow Mountain, Paracas and Huacachina. We will be hiking, visiting museums, and dining out. "

In [24]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [25]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...

Thought: I need to gather information about the trip and the destinations we will be visiting.
Action: Search
Action Input: "Peru travel information"
Observation: [{'title': 'Historic Sanctuary of Machu Picchu', 'link': 'https://www.google.com/search?sca_esv=dec68b1769b74fc5&gl=us&hl=en&q=Historic+Sanctuary+of+Machu+Picchu&si=AMgyJEvmed8FkyEkpEJ8jfGhZkakcy5kQho_c4G-QJRdklshMsXBpT79QPGt_AG4hUy-jSR2NR4SB-qGB7IxPfMxNv_MAkvCJw%3D%3D&sa=X&ved=2ahUKEwjWspO6xLqOAxUuH0QIHVMWLCgQs4ILegQIeBAD', 'description': 'Inca citadel with Temple of the Sun', 'flight_price': '$711', 'extracted_flight_price': 711, 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTIZtT8G4vrV1uXd2VVoGecqqKnFUu9AeLhi-afuhZhStEH0WYWNOgR0yRnTvA&s=8'}, {'title': 'Lima', 'link': 'https://www.google.com/search?sca_esv=dec68b1769b74fc5&gl=us&hl=en&q=Lima&si=AMgyJEvmed8FkyEkpEJ8jfGhZkakcy5kQho

{'objective': 'Create a preparation list From documents needed down to luggage items for our July 27 to August 7, 2025 Trip to Peru. We will be visiting Lima, Cusco, Machu Picchu, Rainbow Mountain, Paracas and Huacachina. We will be hiking, visiting museums, and dining out. '}

When prompt is not changed and only objective has been revised

In [22]:
OBJECTIVE = "Create a preparation list From documents needed down to luggage items for our July 27 to August 7, 2025 Trip to Peru. We will be visiting Lima, Cusco, Machu Picchu, Rainbow Mountain, Paracas and Huacachina. We will be hiking, visiting museums, and dining out. "

In [24]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [25]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to gather all the necessary information and organize it into a list
Action: TODO
Action Input: Create a preparation list for our July 27 to August 7, 2025 Trip to Peru
Observation: 

1. Research and decide on the specific cities and attractions to visit in Peru.
2. Determine the best mode of transportation for traveling within Peru.
3. Book flights and accommodations for the trip.
4. Apply for necessary travel documents, such as passports and visas.
5. Create a budget for the trip, including expenses for transportation, accommodations, food, and activities.
6. Research and purchase travel insurance.
7. Plan and book any necessary tours or activities in advance.
8. Make a packing list and start gathering necessary items, such as appropriate clothing and travel gear.
9. Research and familiarize yourself with the local customs and culture of Peru.
10.

{'objective': 'Create a preparation list From documents needed down to luggage items for our July 27 to August 7, 2025 Trip to Peru. We will be visiting Lima, Cusco, Machu Picchu, Rainbow Mountain, Paracas and Huacachina. We will be hiking, visiting museums, and dining out. '}

changing objectives, prompts and tools

In [6]:
from langchain_openai import OpenAIEmbeddings

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
# vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})



In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool
from langchain_openai import OpenAI

# 1) VisaPreparationAgent
visa_prompt = PromptTemplate.from_template(
    """You are a travel-visa specialist.
Trip objective: {objective}

Using ONLY the tools VisaDocs and Search, list all visa & entry requirements
(forms, fees, timelines) for each country in the trip.  
Return a JSON array of objects with fields:
  - country
  - requirement
  - link
"""
)
visa_chain = LLMChain(llm=OpenAI(temperature=0), prompt=visa_prompt)
VisaPreparationAgent = Tool(
    name="VisaPreparationAgent",
    func=lambda objective: visa_chain.run(objective=objective),
    description="Compile visa & entry requirements for the trip."
)


# 2) PackingListAgent
packing_prompt = PromptTemplate.from_template(
    """You are a packing-list expert.
Trip objective: {objective}
Weather summary: {weather_summary}
Elevation summary: {elevation_summary}

Using ONLY WeatherAPI, ElevationLookup, and Search,
produce a detailed packing checklist grouped by category
(clothing, documents, electronics, medical, misc).
Prioritize items needed for high elevation. Return as bullet points.
"""
)
packing_chain = LLMChain(llm=OpenAI(temperature=0), prompt=packing_prompt)
PackingListAgent = Tool(
    name="PackingListAgent",
    func=lambda objective, weather_summary, elevation_summary: packing_chain.run(
        objective=objective,
        weather_summary=weather_summary,
        elevation_summary=elevation_summary
    ),
    description="Generate the packing list based on weather & elevation."
)


# 3) ItineraryAgent
itinerary_prompt = PromptTemplate.from_template(
    """You are an itinerary planner.
Trip objective: {objective}

Using ONLY DateCalculator and Search, allocate each activity
(hikes, museums, dining) into the date range of the trip.
Return a JSON array of objects:
  - date
  - activity
  - booking_link (if any)
"""
)
itinerary_chain = LLMChain(llm=OpenAI(temperature=0), prompt=itinerary_prompt)
ItineraryAgent = Tool(
    name="ItineraryAgent",
    func=lambda objective: itinerary_chain.run(objective=objective),
    description="Build a day-by-day itinerary for the trip."
)


# 4) BudgetAgent
budget_prompt = PromptTemplate.from_template(
    """You are a budget tracker.
Trip objective: {objective}
Exchange rate (USD→PEN): {exchange_rate}

Using ONLY CurrencyConverter and RecipeSearch, estimate total trip cost
(lodging, food, transport) and return a JSON report:
  - category
  - estimated_cost_usd
  - remaining_budget_usd
"""
)
budget_chain = LLMChain(llm=OpenAI(temperature=0), prompt=budget_prompt)
BudgetAgent = Tool(
    name="BudgetAgent",
    func=lambda objective, exchange_rate: budget_chain.run(
        objective=objective, exchange_rate=exchange_rate
    ),
    description="Estimate trip expenses and compare to budget."
)


# 5) ResultValidator
validate_prompt = PromptTemplate.from_template(
    """You are a trip-plan validator.
Packing list: {packing_list}
Itinerary: {itinerary}
Visa requirements: {visa_requirements}
Budget report: {budget_report}

Check that these essentials are covered: passport, tickets, travel insurance,
altitude medication, local emergency contacts.  
Return a bullet list of missing items, or 'All good.' if none.
"""
)
validate_chain = LLMChain(llm=OpenAI(temperature=0), prompt=validate_prompt)
ResultValidator = Tool(
    name="ResultValidator",
    func=lambda packing_list, itinerary, visa_requirements, budget_report: validate_chain.run(
        packing_list=packing_list,
        itinerary=itinerary,
        visa_requirements=visa_requirements,
        budget_report=budget_report
    ),
    description="Check that all critical trip items are present; list any omissions."
)


C:\Users\happy\AppData\Local\Temp\ipykernel_98632\3154693548.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  visa_chain = LLMChain(llm=OpenAI(temperature=0), prompt=visa_prompt)


In [8]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a senior travel-logistics specialist.  "
    "Your goal is to build a comprehensive preparation checklist—from gathering documents through packing list—for this trip:\n\n"
    "{objective}\n\n"
    "Produce a step-by-step, prioritized task list (numbered)."
)

todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    VisaPreparationAgent,
    PackingListAgent,
    ItineraryAgent,
    BudgetAgent,
    ResultValidator,
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
    # Tool("VisaDocs", VisaPreparationAgent.run, "Compile visa & entry requirements"),
    # Tool("Packing", PackingListAgent.run, "Generate packing checklist based on weather & altitude"),
    # Tool("Itinerary", ItineraryAgent.run, "Build day-by-day schedule"),
    # Tool("Budget", BudgetAgent.run, "Track expenses against budget"),
    # Tool("Validate", ResultValidator.run, "Check that outputs cover all trip needs"),
]


prefix = """You are a Travel-Preparation AI that helps plan every step for a complex trip.
Objective: {objective}
Previously completed tasks & findings:
{context}

Use only the tools available to you to accomplish the next task."""
suffix = """Next task: {task}

Begin thinking (you can call tools like Search or TODO):
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)


In [9]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

C:\Users\happy\AppData\Local\Temp\ipykernel_98632\3064747660.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


In [10]:
OBJECTIVE = "Create a preparation list From documents needed down to luggage items for our July 27 to August 7, 2025 Trip to Peru. We will be visiting Lima, Cusco, Machu Picchu, Rainbow Mountain, Paracas and Huacachina. We will be hiking, visiting museums, and dining out. We are Dutch citizens.Flights, hotels and tours are already booked."

In [11]:
memory = vectorstore.as_retriever(search_kwargs={"k": 3})
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations
)

In [12]:
baby_agi({"objective": OBJECTIVE})

C:\Users\happy\AppData\Local\Temp\ipykernel_98632\3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...

Thought: I need to create a todo list for the objective of preparing for a trip to Peru from July 27 to August 7, 2025.
Action: TODO
Action Input: Prepare for trip to Peru from July 27 to August 7, 2025
Observation: 

1. Check passport expiration date and renew if necessary.
2. Research visa requirements for Peru and apply if needed.
3. Book flights to Peru for July 27 and return on August 7.
4. Research and book accommodations for the duration of the trip.
5. Check for any required vaccinations for travel to Peru and schedule appointments if needed.
6. Purchase travel insurance.
7. Create a budget for the trip and allocate funds for transportation, accommodations, activities, and meals.
8. Research and plan activities and excursions for the trip.
9. Make a list of important contact information, including emergency contacts, embassy information, and hotel informa

C:\Users\happy\AppData\Local\Temp\ipykernel_98632\3154693548.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  func=lambda objective: visa_chain.run(objective=objective),



Observation:   - form
  - fee
  - timeline


[
  {
    "country": "Netherlands",
    "requirement": "Residence permit for the purpose of naturalization",
    "link": "https://www.netherlandsandyou.nl/travel-and-residence/visas-for-the-netherlands/residence-permit-for-the-purpose-of-naturalisation",
    "form": "Application form for residence permit",
    "fee": "€171",
    "timeline": "Processing time of 90 days"
  },
  {
    "country": "Peru",
    "requirement": "Tourist visa",
    "link": "https://www.embassyofperu.org/visas",
    "form": "Online application form",
    "fee": "US$30",
    "timeline": "Processing time of 5-10 business days"
  }
]
Thought: I should also check if there are any additional entry requirements for Peru, such as proof of yellow fever vaccination.
Action: Search
Action Input: "Peru entry requirements"
Observation: ['Peru requires everyone entering the country to have passports valid for at least six months from the date of entry into Peru. · Your approved ..

{'objective': 'Create a preparation list From documents needed down to luggage items for our July 27 to August 7, 2025 Trip to Peru. We will be visiting Lima, Cusco, Machu Picchu, Rainbow Mountain, Paracas and Huacachina. We will be hiking, visiting museums, and dining out. We are Dutch citizens.Flights, hotels and tours are already booked.'}